In [ ]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt

from scipy import stats
from scipy.special import expit as logistic
from scipy.special import softmax

%config InlineBackend.figure_format = 'retina'
warnings.simplefilter(action="ignore", category=FutureWarning)
RANDOM_SEED = 8927
np.random.seed(286)

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### 1. Career choices

In [ ]:
# simulate career choices among 500 individuals
N = 500  # number of individuals
income = np.array([1, 2, 5])  # expected income of each career
score = 0.5 * income  # score for each career, based on income
# converts scores to probabilities:
p = softmax(score)

# now simulate choice
# outcome career holds event type values, not counts
career = np.random.multinomial(1, p, size=N)3
career = np.where(career == 1)[1]
career[:11], score, p

(array([2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2]),
 array([0.5, 1. , 2.5]),
 array([0.09962365, 0.16425163, 0.73612472]))

In [ ]:
with pm.Model() as m1_1:
    a = pm.Normal("a", 0, 1, shape=2)
    b = pm.HalfNormal("b", 0.5)
    
    income_0 = pm.Data("income_0", value=1)
    income_1 = pm.Data("income_1", value=2)
    income_2 = pm.Data("income_2", value=5)
    
    s0 = a[0] + b * income_0
    s1 = a[1] + b * income_1
    s2 = 0.0 + b * income_2
    
    s = pm.math.stack([s0, s1, s2])
    p_ = pm.Deterministic("p", tt.nnet.softmax(s))
    
    career_obs = pm.Categorical('career', p=p_, observed=career)
    trace1_1 = pm.sample(tune=2000, target_accept=0.99, random_seed=RANDOM_SEED)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.
  
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a]


Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000 draws total) took 16 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [ ]:
az.summary(trace1_1)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a[0],-0.288,0.727,-1.676,0.950,0.033,0.023,479.0,479.0,477.0,544.0,1.0
a[1],-0.083,0.552,-1.126,0.883,0.025,0.018,477.0,477.0,478.0,524.0,1.0
b,0.470,0.182,0.150,0.807,0.008,0.006,482.0,460.0,481.0,597.0,1.0
"p[0,0]",0.086,0.013,0.061,0.110,0.000,0.000,3242.0,3242.0,3182.0,2300.0,1.0
"p[0,1]",0.168,0.017,0.137,0.199,0.000,0.000,2776.0,2776.0,2764.0,2608.0,1.0
"p[0,2]",0.746,0.020,0.709,0.782,0.000,0.000,3728.0,3728.0,3718.0,2512.0,1.0


In [ ]:
p_org = trace1_1["p"]

##### ASSUMPTION: When income_1 is double

In [ ]:
with m1_1:
    pm.set_data({"income_1": 4})
    ppc = pm.sample_posterior_predictive(trace1_1, var_names=["a", "p"])

In [ ]:
p_new = ppc["p"]

In [ ]:
p_diff = p_new - p_org
az.summary({"p_diff": p_diff}, kind="stats")

,mean,sd,hdi_3%,hdi_97%
p_diff[0],-0.018,0.019,-0.054,0.017
p_diff[1],0.177,0.085,0.025,0.335
p_diff[2],-0.159,0.077,-0.302,-0.020


In [ ]:
type(trace1_1), type(ppc)

(pymc3.backends.base.MultiTrace, dict)